# component.packet

> TODO fill in description

In [1]:
#| default_exp comp.packet

In [2]:
#| hide
from nbdev.showdoc import *; 

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
from __future__ import annotations
from typing import Any, Optional
from datetime import datetime
from datetime import datetime, timezone
import uuid

import fbdev
from fbdev._utils import SingletonMeta

ImportError: cannot import name '_utils' from partially initialized module 'fbdev' (most likely due to a circular import) (/Users/lukastk/proj_dev/fbdev/fbdev/__init__.py)

In [ ]:
#|hide
show_doc(fbdev.comp.packet.Packet)

---

### Packet

>      Packet (data:Any)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Packet:
    _NullPayload = SingletonMeta('NullPayload')
    
    def __init__(self, data:Any):
        self._uuid:int = uuid.uuid4().int
        self._timestamp:datetime = datetime.now(timezone.utc)
        self._data:Any = data
        self._dtype:type = type(data)
        self._is_consumed:bool = False
        self._consumed_timestamp:Optional[datetime] = None
            
    @property
    def uuid(self): return self._uuid
    @property
    def timestamp(self): return self._timestamp
    @property
    def is_empty(self): return self._dtype == Packet._NullPayload
    @property
    def dtype(self): return self._dtype
    @property
    def is_consumed(self): return self._is_consumed
    @property
    def is_unregistered(self): return self._unregistered
    
    async def consume(self):
        if self._is_consumed: raise RuntimeError("Packet is already consumed.")
        self._is_consumed = True
        self._is_consumed = datetime.now(timezone.utc)
        return self._data
    
    async def peek(self):
        if self._is_consumed: raise RuntimeError("Packet is already consumed.")
        return self._data
    
    @classmethod
    def get_empty(cls):
        return cls(Packet._NullPayload())